# Programa auxiliar para separar imágenes de Entrenamiento y Prueba distribuyendolas al azar

In [ ]:
#@title Librerías a usar
import os
import os.path
import numpy as np
import shutil

print("\nLibrerías importadas")

In [ ]:
#@title Acceder al Drive
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = '/content/gdrive/My Drive/IA/demoML/imagenes' #@param {type:"string"}
path_fuente = '/original'  #@param {type:"string"}
path_destino_entrenamiento = '/train'  #@param {type:"string"}
path_destino_prueba = '/test'  #@param {type:"string"}

imagPath = path + path_fuente
imagPath_train = path + path_destino_entrenamiento
imagPath_test = path + path_destino_prueba

In [ ]:
#@title Levantar carpeta de imágenes a distribuir

# procesa los archicos
todasImags = {}

print("Procesando ",imagPath,"...\n")

all_dirs = os.listdir( imagPath )
for clase in all_dirs:

    auxiPath = imagPath + '/' + clase

    if os.path.isdir( auxiPath ):
      # levanta la lista de imágenes de cada letra
      todasImags[clase] = [ fn for fn in os.listdir( auxiPath ) if fn.endswith('.png') or fn.endswith('.jpg') or fn.endswith('.PRN') or fn.endswith('.JPG') or fn.endswith('.png') or fn.endswith('.jpeg') or fn.endswith('.JPEG') ]
      # desordena al azar
      np.random.shuffle( todasImags[clase] )

      print("Levanta de ", clase, ": ", len(todasImags[clase])," imágenes.")



In [ ]:
#@title Normaliza cantidades por clase (opcional)
unificar_cantid_por_clase = False #@param {type:"boolean"}

if unificar_cantid_por_clase:
  # determina cantidad mínima
  min = None
  for clase in todasImags.keys():
     cantImg = len(todasImags[clase])
     if (min is None) or (min > cantImg):
       if cantImg>0:
          min = cantImg

  print("Cantidad a unificar: ", min)
  print("\n")
  # unifica esa cantidad como mínima
  for clase in todasImags.keys():
    todasImags[clase] = np.random.choice(todasImags[clase], size=min, replace=False)

    print("Nueva cantidad de ", clase, ": ", len(todasImags[clase])," imágenes.")


In [ ]:
#@title Determina Cantidades a distribuir

porc_Entrenamiento = 80 #@param {type:"slider", min:10, max:100, step:1}
if porc_Entrenamiento < 10:
  porc_Entrenamiento = 10

# muestra proporciones a usar
for clase in todasImags.keys():
  cantImg = len(todasImags[clase])
  cantE = ( cantImg * porc_Entrenamiento ) // 100
  cantP = cantImg - cantE

  print('\nImágenes de ' , clase, ":")
  print('- Imágenes para Entrenamiento: ', cantE)
  print('- Imágenes para Validación: ', cantP)

In [ ]:
#@title Mueve archivos para Entrenamiento

def moverArchivos(imagPathOrigen_Clase, arFNClase, imagPathDestino_Clase):
  cant = 0
  for name in arFNClase:
      # mueve la imagen
      if os.path.isfile(imagPathDestino_Clase + '/' + name):
        print("**", name, " ya existe en ", imagPathDestino_Clase)
      else:
        shutil.move(imagPathOrigen_Clase + '/' + name, imagPathDestino_Clase)
        cant = cant + 1
  print("\t", cant, " imágenes movidas a ", imagPathDestino_Clase)
  return cant


# crea directorios, si es necesario
if not os.path.isdir(imagPath_train):
    os.makedirs(imagPath_train)
if not os.path.isdir(imagPath_test):
    os.makedirs(imagPath_test)

for clase in todasImags.keys():

  print("> ", clase, ":")
  cantE = 0
  cantP = 0
  # crea subdirectorios, si es necesario
  imagpathOrigenClase = imagPath + '/' + clase
  imagPath_trainClase = imagPath_train + '/' + clase
  imagPath_testClase = imagPath_test + '/' + clase
  if not os.path.isdir(imagPath_trainClase):
      os.makedirs(imagPath_trainClase)
  if not os.path.isdir(imagPath_testClase):
      os.makedirs(imagPath_testClase)

  # determina cantidades
  arImgsFN = np.array(todasImags[clase])
  cantImg = len(todasImags[clase])
  cantE = ( cantImg * porc_Entrenamiento ) // 100
  #cantP = cantImg - cantE

  # separa
  train_FileNames, test_FileNames = np.split(arImgsFN, [cantE])

  # mueve imágenes
  moverArchivos(imagpathOrigenClase, train_FileNames, imagPath_trainClase)
  moverArchivos(imagpathOrigenClase, test_FileNames, imagPath_testClase)


In [ ]:
#@title Fuerza actualización del drive (sino no aparecen los nuevos archivos)
drive.flush_and_unmount()
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
#@title Muestra nuevas cantidades
def contarImagenes(imagPath):

  print("\n\n> procesando ", imagPath, ": ")

  # procesa los subdirectorios con las imágenes
  all_dirs = os.listdir( imagPath )
  for each_dir in all_dirs:

    auxiPath = imagPath + '/' + each_dir

    # obtiene la cantidad de imágenes disponibles
    cantImagDir = len(os.listdir( auxiPath ))

    print("  -- ", auxiPath, ": tiene ", cantImagDir, " imágenes.")

# realiza conteo
contarImagenes(imagPath_train)
contarImagenes(imagPath_test)